In [70]:
import trimesh
import numpy as np
import k3d

import polars as pl

from pathlib import Path

import os
import glob
import ipywidgets as widgets
from IPython.display import display
from tqdm import tqdm



In [71]:
df = pl.read_csv("ShapenetSem/metadata.csv")


categories = "(?i)bed|chair|couch|desk|table|lamp|shelf|wardrobe"
result_df = df.filter(
    (pl.col("category").str.contains(categories)) & 
    (pl.col("unit").is_not_null())   # skip models without units
)


result_df

fullId,category,wnsynset,wnlemmas,up,front,unit,aligned.dims,isContainerLike,surfaceVolume,solidVolume,supportSurfaceArea,weight,staticFrictionForce,name,tags
str,str,str,str,str,str,f64,str,str,str,str,str,str,str,str,str
"""wss.100f39dce7690f59efb94709f3…","""Chair,Recliner""","""n4069540""","""recliner,reclining chair,loung…","""0\,0\,1""","""0\,-1\,0""",0.012947,"""111.34567\,100.547745\,96.1327…",null,null,null,null,null,null,"""couch""","""carpet recliner"""
"""wss.1022fe7dd03f6a4d4d5ad9f13a…","""Chair,OfficeChair""","""n3005231""","""chair""","""0\,0\,1""","""0\,1\,0""",0.017984,"""60.366123\,98.00925\,66.79712""",null,null,null,null,null,null,"""office chair""","""fauteuil de bureau,office chai…"
"""wss.1028b32dc1873c2afe26a3ac36…","""Chair,OfficeSideChair""","""n3005231""","""chair""","""0\,0\,1""","""0\,-1\,0""",0.036115,"""78.41001\,117.685616\,77.54468""",null,null,null,null,null,null,"""visavis stackable chair""","""chair office designer citterio"""
"""wss.102a6b7809f4e51813842bc8ef…","""Computer,Desktop""","""n3184677""","""desktop computer""",null,null,0.025252,"""23.257397\,46.2361\,60.778683""",null,null,null,null,null,null,"""alienware area computer cas…","""*"""
"""wss.1033ee86cc8bac4390962e4fb7…","""Chair""","""n2741540""","""armchair""","""0\,0\,1""","""-1\,0\,0""",0.024691,"""66.430275\,98.00357\,109.57538""",null,null,null,null,null,null,"""the armchair cit eacute""","""armchair,chair,chaise,french,f…"
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""wss.f69510412f10e9ed322ed2ef5f…","""Lamp,WallLamp""","""n4155768""","""sconce""","""0\,0\,1""","""-1\,0\,0""",0.059747,"""42.131245\,90.37565\,36.08628""",null,null,null,null,null,null,"""antique lighting sconce""","""antique,double e,light,old,sco…"
"""wss.738cebb6eab9151bd4a8d9e46a…","""Lamp,WallLamp""","""n3670692""","""light,light source""","""0\,0\,1""","""1\,0\,0""",0.08408,"""51.64453\,84.50152\,37.83612""",null,null,null,null,null,null,"""vaporproof wall light jelly j…","""jelly jar,light,vapor,vaporpro…"
"""wss.571a3c2cf4c509596cb5b29312…","""Lamp,WallLamp""","""n4155768""","""sconce""","""0\,1\,0""","""0\,0\,1""",0.069445,"""69.44497\,39.93086\,69.44497""",null,null,null,null,null,null,"""sconce round""","""contemporary,lamp,light,lighti…"


In [77]:
# Find indices of models where up vector is NOT Z-up (0,0,1)
non_z_up_indices = [i for i, row in enumerate(result_df.iter_rows(named=True)) 
                   if row['up'] and row['up'] != "0\\,0\\,1"]
print(f"Models with non Z-up: {len(non_z_up_indices)} found")
print(f"Slider indices: {non_z_up_indices}")

Models with non Z-up: 37 found
Slider indices: [25, 105, 276, 318, 355, 524, 591, 630, 712, 745, 768, 802, 859, 939, 951, 975, 1019, 1379, 1564, 1713, 1835, 1840, 1936, 2143, 2439, 2471, 2683, 2688, 2693, 2710, 2770, 2771, 2772, 2776, 2777, 2783, 2784]


In [ ]:
model_path = Path("ShapenetSem/models-COLLADA/COLLADA/")

def parse_axis(axis_str):
    """Parse axis string like '0\\,0\\,1' to numpy array."""
    parts = axis_str.replace("\\", "").split(",")
    return np.array([float(p) for p in parts])

def format_axis(axis_str):
    """Format axis string with 2 decimal places."""
    if not axis_str:
        return 'N/A'
    arr = parse_axis(axis_str)
    return f"({arr[0]:.2f}, {arr[1]:.2f}, {arr[2]:.2f})"

def apply_rotations(mesh, up_str, front_str):
    """Rotate mesh so that up -> Z+ and front -> X+."""
    up = parse_axis(up_str)
    front = parse_axis(front_str)
    
    # Normalize
    up = up / np.linalg.norm(up)
    front = front / np.linalg.norm(front)
    
    # Build source basis (front, right, up)
    right = np.cross(up, front)
    right = right / np.linalg.norm(right)
    
    # Recompute front to ensure orthogonality
    front = np.cross(right, up)
    front = front / np.linalg.norm(front)
    
    # Source basis matrix (columns are basis vectors)
    src_basis = np.column_stack([front, right, up])
    
    # Target basis: X+=front, Y+=right, Z+=up (identity)
    # Rotation matrix: R @ src = target, so R = target @ src^-1 = src.T (orthonormal)
    rotation_matrix = src_basis.T
    
    mesh.vertices = mesh.vertices @ rotation_matrix.T
    return mesh

# Create k3d plot with empty mesh and label
plot = k3d.plot(camera_mode="orbit", grid_auto_fit=False, grid=[-2, -2, 0, 2, 2, 2])
mesh_obj = k3d.mesh(np.zeros((0, 3), dtype=np.float32), np.zeros((0, 3), dtype=np.uint32), color=0x00FF00)
label = k3d.text2d("", position=(0.02, 0.95), size=1.0, color=0x000000)
plot += mesh_obj
plot += label
plot.display()

def update_mesh(index, apply_rotation):
    """Update the displayed mesh based on slider index."""
    row = result_df.row(index, named=True)
    fullid = row["fullId"][4:]
    mesh_file = model_path / f"{fullid}.dae"
    
    try:
        mesh = trimesh.load_mesh(mesh_file)
        
        # Apply unit scale if available
        if row['unit'] is not None:
            mesh.apply_scale(float(row['unit']))
        
        # Apply rotation if checkbox is checked and up/front are available
        if apply_rotation and row['up'] and row['front']:
            mesh = apply_rotations(mesh, row['up'], row['front'])
        
        mesh_obj.vertices = mesh.vertices.astype(np.float32)
        mesh_obj.indices = mesh.faces.astype(np.uint32)
        
        up_val = format_axis(row['up'])
        front_val = format_axis(row['front'])
        unit_val = f"{float(row['unit']):.2f}" if row['unit'] else 'N/A'
        label.text = f"up: {up_val}  |  front: {front_val}  |  unit: {unit_val}"
        
        print(f"Model {index}: {row['name']} ({row['category']})")
    except Exception as e:
        print(f"Failed to load {fullid}: {e}")

slider = widgets.IntSlider(
    value=0, min=0, max=len(result_df) - 1, step=1,
    description='Model:', continuous_update=False,
    layout=widgets.Layout(width='100%')
)

checkbox = widgets.Checkbox(
    value=False,
    description='Apply Z-up / X-front rotation',
    indent=False
)

widgets.interact(update_mesh, index=slider, apply_rotation=checkbox)

In [80]:
# filter categories



categories = "(?i)bed|chair|couch|desk|table|lamp|shelf|wardrobe"
result_df = df.filter(
    (pl.col("category").str.contains(categories)) & 
    (pl.col("unit").is_not_null())   # skip models without units
)

output_df = result_df.select([
    # 1. Remove first 4 characters from fullId
    pl.col("fullId").str.slice(4),
    
    # 2. Process category column
    pl.col("category")
    .str.split(",")                           # Split into a list of words
    .list.eval(                               # Evaluate each item in the list
        pl.element().filter(
            # Keep elements that do NOT start with "_" (after trimming spaces)
            ~pl.element().str.strip_chars().str.starts_with("_")
        )
    )
    .list.join(",")                           # Join the remaining words back
])

# Write to CSV
output_df.write_csv("inventory.csv")

# Optional: Print to verify
print(output_df.head())

shape: (5, 2)
┌─────────────────────────────────┬───────────────────────┐
│ fullId                          ┆ category              │
│ ---                             ┆ ---                   │
│ str                             ┆ str                   │
╞═════════════════════════════════╪═══════════════════════╡
│ 100f39dce7690f59efb94709f30ce0… ┆ Chair,Recliner        │
│ 1022fe7dd03f6a4d4d5ad9f13ac9f4… ┆ Chair,OfficeChair     │
│ 1028b32dc1873c2afe26a3ac360dbd… ┆ Chair,OfficeSideChair │
│ 102a6b7809f4e51813842bc8ef6fe1… ┆ Computer,Desktop      │
│ 1033ee86cc8bac4390962e4fb7072b… ┆ Chair                 │
└─────────────────────────────────┴───────────────────────┘


In [ ]:
# convert the collada files to glb 
# we work with result_df

def parse_axis(axis_str):
    """Parse axis string like '0\\,0\\,1' to numpy array."""
    parts = axis_str.replace("\\", "").split(",")
    return np.array([float(p) for p in parts])

def apply_rotations(mesh, up_str, front_str):
    """Rotate mesh so that up -> Z+ and front -> X+."""
    up = parse_axis(up_str)
    front = parse_axis(front_str)
    
    # Normalize
    up = up / np.linalg.norm(up)
    front = front / np.linalg.norm(front)
    
    # Build source basis (front, right, up)
    right = np.cross(up, front)
    right = right / np.linalg.norm(right)
    
    # Recompute front to ensure orthogonality
    front = np.cross(right, up)
    front = front / np.linalg.norm(front)
    
    # Source basis matrix (columns are basis vectors)
    src_basis = np.column_stack([front, right, up])
    rotation_matrix = src_basis.T
    
    mesh.vertices = mesh.vertices @ rotation_matrix.T
    return mesh

model_path = Path("ShapenetSem/models-COLLADA/COLLADA/")
glb_path = Path("GLB/")
glb_path.mkdir(exist_ok=True, parents=True)

for row in tqdm(result_df.iter_rows(named=True), total=len(result_df)):
    full_id = row["fullId"]
    unit = row["unit"]
    
    if unit is None: 
        print(f"Skipping {full_id}: No unit defined")
        continue

    short_id = full_id[4:]
    source_file = model_path / f"{short_id}.dae"
    dest_file = glb_path / f"{short_id}.glb"

    try:
        tm_mesh = trimesh.load(source_file, force='mesh')
        tm_mesh.apply_scale(float(unit))
        
        # Apply rotation if up and front are available
        if row['up'] and row['front']:
            tm_mesh = apply_rotations(tm_mesh, row['up'], row['front'])
        
        tm_mesh.export(dest_file)
        
    except Exception as e:
        print(f"Failed to process {short_id}: {e}")

print("Processing complete.")

 38%|██████████████▊                        | 1058/2786 [02:10<03:08,  9.19it/s]